In [9]:
import pyarrow as pa
from datasets import load_dataset
import random

# Dataset Tests

In [40]:
import os

In [ ]:
current_dir = os.path.dirname(os.path.abspath(__file__))

In [41]:
parent_dir = os.path.join(current_dir, '..')

NameError: name 'current_dir' is not defined

NameError: name '__file__' is not defined

In [33]:
DATASET= 'TheFinAI/FINQA_test'
DEFAULT_SUBSET = 140

In [16]:
ds = load_dataset(DATASET, split='test')

In [17]:
ds

Dataset({
    features: ['id', 'Open-ended Verifiable Question', 'Ground-True Answer'],
    num_rows: 1147
})

In [34]:
indices = list(range(len(ds)))
random.shuffle(indices)

In [35]:
n=DEFAULT_SUBSET

In [36]:
indices = indices[:n]

In [21]:
ds[indices[0]]

{'id': 'FINQA707',
 'Open-ended Verifiable Question': 'Please answer the given financial question based on the context.\nContext: value using an appropriate discount rate . projected cash flow is discounted at a required rate of return that reflects the relative risk of achieving the cash flow and the time value of money . the market approach is a valuation technique that uses prices and other relevant information generated by market transactions involving identical or comparable assets , liabilities , or a group of assets and liabilities . valuation techniques consistent with the market approach often use market multiples derived from a set of comparables . the cost approach , which estimates value by determining the current cost of replacing an asset with another of equivalent economic utility , was used , as appropriate , for property , plant and equipment . the cost to replace a given asset reflects the estimated reproduction or replacement cost for the property , less an allowance

In [22]:
ds[indices[0]].keys()

dict_keys(['id', 'Open-ended Verifiable Question', 'Ground-True Answer'])

In [23]:
row = ds[indices[0]]

In [24]:
q = row["Open-ended Verifiable Question"]

In [25]:
best = row['Ground-True Answer']

In [26]:
best

'0.04171'

In [38]:
for i in range(100):
    print(ds[indices[i]]['Ground-True Answer'])

-0.0365
0.4
0.27404
91.8
0.0641
0.53825
10626.0
-29.0
0.12204
30.87736
28000.0
2e-05
0.11212
117624.84
0.70203
0.17706
0.84509
148.36
yes
172.2
33458.0
150.0
0.45861
0.19391
87.8
33.33333
209703.0
no
0.608
0.26341
27.80639
0.01741
0.26856
1723732.78
0.33333
0.41467
-0.55319
0.68343
-13636.0
0.81238
708.1
0.03816
0.74953
240.41
0.21864
0.27273
0.29944
-3155.0
0.15653
17.0
yes
0.16498
0.88928
0.09864
0.379
0.2941
0.45103
0.20904
1.5
0.52946
-35.0
0.07166
4781374.0
0.35908
-305.0
0.18186
0.0473
0.166
0.57534
0.02
0.10482
0.14167
2.93333
52336.0
448.0
0.67877
0.11713
1214.66667
5.9
0.07288
74161.5
0.36161
0.16534
0.1954
1.26207
0.11797
0.80473
-0.17521
0.02652
0.16127
1056.0
-3813.0
0.55049
2.0
160035.0
0.29167
26.42857
1.13599
0.81572
-13636.0


In [28]:
# prompts = []
# refs = []
# for i, idx in enumerate(indices):
#     row = ds[int(idx)]
#     q = row['data']["Question"]
#     best = row['data']['Options'][row['data']["Correct Option"]]
#     rid = f"truth-{i:04d}"
#     prompts.append({
#             "id": rid,
#             "prompt": q,
#             "system": "You are a concise, truthful assistant. Answer briefly and accurately."
#         })
#     refs.append({"id": rid, "references": best, "reference": best[0]})

# Inference Testing

In [1]:
import json, urllib.request

In [3]:
MODEL_OLLAMA = "llama2:7b"
SEED = 42

In [4]:
model: str = MODEL_OLLAMA
temperature: float = 0.3
top_p: float = 0.9
max_tokens: int = 256
seed: int = SEED

In [36]:
prompt= "Trilochanapala, last king of the Kabul Shahi dynasty, ruled in the site of which modern country?"

In [37]:
req = urllib.request.Request(
    "http://localhost:11434/api/generate",
    data=json.dumps({
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens,
        "seed": seed,
        "stream": False
    }).encode("utf-8"),
    headers={"Content-Type": "application/json"}
)

In [38]:
try:
    with urllib.request.urlopen(req, timeout=600) as resp:
        out = json.loads(resp.read().decode("utf-8"))
        response = out.get("response", "").strip()
except Exception as e:
    die(f"Ollama HTTP call failed. Is 'ollama serve' running? Error: {e}")

In [39]:
response

'The Trilochanapala was a ruler of the ancient Kabul Shahi dynasty, which is believed to have ruled in the region of modern-day Afghanistan. Specifically, the Kabul Shahi kingdom was centered around the city of Kabul and the surrounding areas. So, the answer is Afghanistan.'

In [40]:
CONFIDENCE_QUESTION = "Given the question and your answer, how confident are you that you are correct. Answer in exactly one word from [High, Med, Low]"

In [41]:
confidence_prompt = f"QUESTION:\n{prompt}\nYOU RESPONSE:\n{response}\n\n{CONFIDENCE_QUESTION}"

In [42]:
confidence_prompt

'QUESTION:\nTrilochanapala, last king of the Kabul Shahi dynasty, ruled in the site of which modern country?\nYOU RESPONSE:\nThe Trilochanapala was a ruler of the ancient Kabul Shahi dynasty, which is believed to have ruled in the region of modern-day Afghanistan. Specifically, the Kabul Shahi kingdom was centered around the city of Kabul and the surrounding areas. So, the answer is Afghanistan.\n\nGiven the question and your answer, how confident are you that you are correct. Answer in exactly one word from [High, Med, Low]'

In [43]:
req2 = urllib.request.Request(
    "http://localhost:11434/api/generate",
    data=json.dumps({
        "model": model,
        "prompt": confidence_prompt,
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens,
        "seed": seed,
        "stream": False
    }).encode("utf-8"),
    headers={"Content-Type": "application/json"}
)

try:
    with urllib.request.urlopen(req2, timeout=600) as resp:
        out = json.loads(resp.read().decode("utf-8"))
        score = out.get("response", "").strip()
except Exception as e:
    die(f"Ollama HTTP call failed. Is 'ollama serve' running? Error: {e}")

In [44]:
score

'Med'